<a href="https://colab.research.google.com/github/pushkal1234/Topic-Recommendation-System_E-learning-Platform/blob/main/Topic_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


We will be working with SVD Algorithm **Surprise Singular Value Decomposition (SVD)**


Optimizer:- **Stochastic Gradient Descent**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

We will use the ExamLounge dataset, which is composed of integer ratings from 1 to 5.
**Surprise** supports dataframes as long as they have three colums reprensenting the **user ids, item ids, and the Levels** (in this order).

In [ ]:
import sys
sys.path.append("../../")
import os
!{sys.executable} -m pip install surprise
!{sys.executable} -m pip install papermill 
import papermill as pm
!{sys.executable} -m pip install scrapbook
import scrapbook as sb
import pandas as pd
!{sys.executable} -m pip install reco_utils
from reco_utils.common.timer import Timer
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.recommender.surprise.surprise_utils import predict, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

ERROR: Could not find a version that satisfies the requirement reco_utils (from versions: none)
ERROR: No matching distribution found for reco_utils


ModuleNotFoundError: ignored

In [ ]:
# Select data size: 100, 1k, 2k, or 5k
ExamLounge_DATA_SIZE = '100'     

**Load Data**

In [ ]:
data = movielens.load_pandas_df(
    size=ExamLounge_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

NameError: ignored

**Train the SVD Model**
Splitting our data using into Test set, Train Set using python_random_split
 

In [ ]:
train, test = python_random_split(data, 0.75)

In [ ]:

train_set = surprise.Dataset.load_from_df(train, reader=surprise.Reader('ml-100k')).build_full_trainset()
train_set

In [ ]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=30, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

**Predictions**

In [ ]:
predictions = predict(svd, test, usercol='userID', itemcol='itemID')
predictions.head()

Remove rated in the top k recommendations
To compute ranking metrics, we need predictions on all user, item pairs. We remove though the items already watched by the user, since we choose not to recommend them again.

In [ ]:

with Timer() as test_time:
    all_predictions = compute_ranking_predictions(svd, train, usercol='userID', itemcol='itemID', remove_seen=True)
    
print("Took {} seconds for prediction.".format(test_time.interval))

In [ ]:

all_predictions.head()

Evaluatation of SVD performance
The SVD algorithm was specifically designed to predict ratings as close as possible to their actual values. In particular, it is designed to have a very low RMSE (Root Mean Squared Error), computed as:

$$\sqrt{\frac{1}{N} \sum(\hat{r_{ui}} - r_{ui})^2}$$
As we can see, the RMSE and MAE (Mean Absolute Error) are pretty low (i.e. good), indicating that on average the error in the predicted ratings is less than 1. The RMSE is of course a bit higher, because high errors are penalized much more.

For comparison with other models, we can also display Top-k and ranking metrics (MAP, NDCG, etc.). however that the SVD algorithm was designed for achieving high accuracy, not for top-rank predictions.

In [ ]:
eval_rmse = rmse(test, predictions)
eval_mae = mae(test, predictions)
eval_rsquared = rsquared(test, predictions)
eval_exp_var = exp_var(test, predictions)

k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)


print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

print('----')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

In [ ]:
# Record results with papermill for tests
sb.glue("rmse", eval_rmse)
sb.glue("mae", eval_mae)
sb.glue("rsquared", eval_rsquared)
sb.glue("exp_var", eval_exp_var)
sb.glue("map", eval_map)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)
sb.glue("train_time", train_time.interval)
sb.glue("test_time", test_time.interval)